In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import time


def setGraph( N, edgeList ):
    W = np.zeros((N,N))
    for e in edgeList:
        i = e[0]
        j = e[1]
        W[i][j] = 1
        W[j][i] = 1       
    return W

def setIncidence( N, edgeList ):
    m = len(edgeList)
    B = np.zeros((N,m))
    for idx, e in enumerate(edgeList):
        i = e[0]
        j = e[1]
        B[i,idx] = 1
        B[j,idx] = -1
    return B

if __name__ == '__main__':
    n = 3 # dimension
    N = 8 # number of robots
    K = 100 # number of iterations
    alpha = 0.1# discretization constant
    var_rel = 1
    var_abs = 0.5
    edgeList = [(0,1),
                (1,2),
                (2,3),
                (3,0),
                (4,5),
                (5,6),
                (6,7),
                (7,4),
                (0,4),
                (1,5),
                (2,6),
                (3,7)]
    W = setGraph(N, edgeList)
    Dout = np.diag(np.sum(W, axis=1))
    L = Dout - W 
    B = setIncidence(N,edgeList)
    B_kron = np.kron(B,np.identity(n))
    
    
    p_inf = np.array([0,0,0, 0,1,0, 1,1,0, 1,0,0, 0,0,1, 0,1,1, 1,1,1, 1,0,1]).T
    y = np.dot(B_kron.T,p_inf) + 0.01*(np.random.rand(n*len(edgeList))-0.5) # relative position measurement
    y_abs = p_inf + 0.01*(np.random.rand(n*N)-0.5) # absolute position measurement 

    #nodeList_abs = [0,1,2,3,4,5,6,7]
    nodeList_abs = [0,1,2,3]
    S = np.zeros((N,N))
    for i in nodeList_abs:
        S[i,i] = 1
    S_kron = np.kron(S, np.identity(n))

In [2]:
    A = np.identity(n*N) - (1/var_rel)*alpha*np.kron(L, np.identity(n)) - (1/var_abs)*alpha*np.dot(S_kron.T,S_kron)
    p_est = np.zeros((n*N,K))
    
    for k in range(K-1): 
        p_est[:,k+1] = np.dot(A,p_est[:,k]) + (1/var_rel)*alpha*np.dot(B_kron,y) + (1/var_abs)*alpha*np.dot(np.dot(S_kron.T, S_kron), y_abs)


In [3]:
np.linalg.norm(p_est[:,K-1]-p_inf)

0.0095988287134466665

In [204]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision=4)
p_est[:,K-1] 

array([-0.0009,  0.0003, -0.0043,  0.0003,  1.0023, -0.005 ,  1.0027,
        1.0038, -0.0043,  0.9985,  0.0023, -0.0032, -0.0038,  0.0006,
        0.9952, -0.0035,  1.0001,  0.9963,  0.9968,  1.0023,  0.9985,
        0.9948, -0.0009,  0.9938])